In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from collections import deque

In [2]:
train = pd.read_csv("train.csv")
sim =  pd.read_csv("sim.csv")
first_data = pd.read_csv('first_data_for_ml.csv', index_col=0)
first_data['D_mean'] = first_data.apply(lambda x: x.D_sum / x.D_count, axis=1)
first_data['K_mean'] = first_data.apply(lambda x: x.K_sum / x.K_count, axis=1)
first_data

class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                         
0        NaN  777.614282  2975.333333  0.261354  189.940639  1218.666667   
1        NaN  379.111613  2749.833333  0.137867  320.586678  1293.166667   
2        NaN  464.316126  2942.166667  0.157814  156.871831  1235.500000   
3        NaN  294.214732  2791.833333  0.105384  211.299259  1292.666667   
4        NaN  820.776927  3009.000000  0.272774  196.406639  1249.500000   
...      ...         ...          ...       ...         ...          ...   
14995    NaN  892.810953  3019.000000  0.295731  194.093758  1227.333333   
14996    NaN  307.139350  2754.333333  0.111511  273.660614  1323.500000   
14997    NaN  367.227889  2820.833333  0.130184  219.240044  1303.333333   
14998    NaN  471.464040  2966.333333  0.158938  215.806324  1298.000000   
14999    NaN  460.232824  2657.666667  0.173172  460.621661  1303.666667   

         K_mean  
id               
0      0.155859  
1      0.247908  
2      0.126970  
3      0.163460  
4      0.157188  
...         ...  
14995  0.158143  
14996  0.206770  
14997  0.168215  
14998  0.166261  
14999  0.353328  

[15000 rows x 7 columns]

In [3]:
pred_classes = pd.DataFrame({'id': np.arange(0, 15000),
                'class': [np.nan] * 15000,
                'D_sum': np.zeros(15000),
                'D_count': np.zeros(15000),
                'D_mean': np.zeros(15000),
                'D_squarred_difference': np.zeros(15000),
                'D_standard_error': np.zeros(15000),
                'D_max': np.zeros(15000),
                'D_min': np.ones(15000),
                'K_sum': np.zeros(15000),
                'K_count': np.zeros(15000),
                'K_mean': np.zeros(15000),
                'K_squarred_difference': np.zeros(15000),
                'K_standard_error': np.zeros(15000),
                'K_max': np.zeros(15000),
                'K_min': np.ones(15000)})
pred_classes

id  class  D_sum  D_count  D_mean  D_squarred_difference  \
0          0    NaN    0.0      0.0     0.0                    0.0   
1          1    NaN    0.0      0.0     0.0                    0.0   
2          2    NaN    0.0      0.0     0.0                    0.0   
3          3    NaN    0.0      0.0     0.0                    0.0   
4          4    NaN    0.0      0.0     0.0                    0.0   
...      ...    ...    ...      ...     ...                    ...   
14995  14995    NaN    0.0      0.0     0.0                    0.0   
14996  14996    NaN    0.0      0.0     0.0                    0.0   
14997  14997    NaN    0.0      0.0     0.0                    0.0   
14998  14998    NaN    0.0      0.0     0.0                    0.0   
14999  14999    NaN    0.0      0.0     0.0                    0.0   

       D_standard_error  D_max  D_min  K_sum  K_count  K_mean  \
0                   0.0    0.0    1.0    0.0      0.0     0.0   
1                   0.0    0.0    1.0    0.0      0.0     0.0   
2                   0.0    0.0    1.0    0.0      0.0     0.0   
3                   0.0    0.0    1.0    0.0      0.0     0.0   
4                   0.0    0.0    1.0    0.0      0.0     0.0   
...                 ...    ...    ...    ...      ...     ...   
14995               0.0    0.0    1.0    0.0      0.0     0.0   
14996               0.0    0.0    1.0    0.0      0.0     0.0   
14997               0.0    0.0    1.0    0.0      0.0     0.0   
14998               0.0    0.0    1.0    0.0      0.0     0.0   
14999               0.0    0.0    1.0    0.0      0.0     0.0   

       K_squarred_difference  K_standard_error  K_max  K_min  
0                        0.0               0.0    0.0    1.0  
1                        0.0               0.0    0.0    1.0  
2                        0.0               0.0    0.0    1.0  
3                        0.0               0.0    0.0    1.0  
4                        0.0               0.0    0.0    1.0  
...                      ...               ...    ...    ...  
14995                    0.0               0.0    0.0    1.0  
14996                    0.0               0.0    0.0    1.0  
14997                    0.0               0.0    0.0    1.0  
14998                    0.0               0.0    0.0    1.0  
14999                    0.0               0.0    0.0    1.0  

[15000 rows x 16 columns]

In [4]:
G = nx.from_pandas_edgelist(sim,source='id1',target='id2',edge_attr='sim')
class_dict = pd.Series(train['class'].values, index=train['id']).to_dict()
nx.set_node_attributes(G, class_dict, 'class')

In [5]:
pred_classes = pred_classes.set_index('id')
pred_classes

class  D_sum  D_count  D_mean  D_squarred_difference  D_standard_error  \
id                                                                              
0        NaN    0.0      0.0     0.0                    0.0               0.0   
1        NaN    0.0      0.0     0.0                    0.0               0.0   
2        NaN    0.0      0.0     0.0                    0.0               0.0   
3        NaN    0.0      0.0     0.0                    0.0               0.0   
4        NaN    0.0      0.0     0.0                    0.0               0.0   
...      ...    ...      ...     ...                    ...               ...   
14995    NaN    0.0      0.0     0.0                    0.0               0.0   
14996    NaN    0.0      0.0     0.0                    0.0               0.0   
14997    NaN    0.0      0.0     0.0                    0.0               0.0   
14998    NaN    0.0      0.0     0.0                    0.0               0.0   
14999    NaN    0.0      0.0     0.0                    0.0               0.0   

       D_max  D_min  K_sum  K_count  K_mean  K_squarred_difference  \
id                                                                   
0        0.0    1.0    0.0      0.0     0.0                    0.0   
1        0.0    1.0    0.0      0.0     0.0                    0.0   
2        0.0    1.0    0.0      0.0     0.0                    0.0   
3        0.0    1.0    0.0      0.0     0.0                    0.0   
4        0.0    1.0    0.0      0.0     0.0                    0.0   
...      ...    ...    ...      ...     ...                    ...   
14995    0.0    1.0    0.0      0.0     0.0                    0.0   
14996    0.0    1.0    0.0      0.0     0.0                    0.0   
14997    0.0    1.0    0.0      0.0     0.0                    0.0   
14998    0.0    1.0    0.0      0.0     0.0                    0.0   
14999    0.0    1.0    0.0      0.0     0.0                    0.0   

       K_standard_error  K_max  K_min  
id                                     
0                   0.0    0.0    1.0  
1                   0.0    0.0    1.0  
2                   0.0    0.0    1.0  
3                   0.0    0.0    1.0  
4                   0.0    0.0    1.0  
...                 ...    ...    ...  
14995               0.0    0.0    1.0  
14996               0.0    0.0    1.0  
14997               0.0    0.0    1.0  
14998               0.0    0.0    1.0  
14999               0.0    0.0    1.0  

[15000 rows x 15 columns]

In [6]:
# Function to perform BFS with custom similarity condition and store edge data
def custom_bfs(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    visited = set()
    queue = deque([start_node])
    visited.add(start_node)
    
    # List to store the results [old_vertice, new_vertice, similarity]
    edge_data = []
    
    while queue:
        node = queue.popleft()
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge (node, neighbor)
            similarity = G[node][neighbor].get('sim', 0)
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (similarity >= similarity_threshold):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    edge_data.append([node, neighbor, similarity])
                    #print(f"  Adding {neighbor} with similarity {similarity}")
            
            elif (condition == '<=') and (similarity <= similarity_threshold):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    edge_data.append([node, neighbor, similarity])
                    #print(f"  Adding {neighbor} with similarity {similarity}")
    
    # Return the list of edges with their similarity values
    return edge_data

# Function to perform BFS and track breadth (level) of each node
def bfs_with_breadth(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    queue = deque([(start_node, 0)])  # Each element in the queue is a tuple: (node, breadth)
    visited = set([start_node])      # Track visited nodes
    levels = {start_node: 0}         # Dictionary to store the level (breadth) of each node
    edge_data = []                   # To store [old_node, new_node, similarity, breadth]
    
    while queue:
        node, breadth = queue.popleft()
        # print(f"Visiting node: {node}, Breadth: {breadth}")
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge
            similarity = G[node][neighbor].get('sim', 0)
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (similarity >= similarity_threshold) or \
               (condition == '<=') and (similarity <= similarity_threshold):
                
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_breadth = breadth + 1
                    levels[neighbor] = new_breadth
                    queue.append((neighbor, new_breadth))
                    edge_data.append([node, neighbor, similarity, new_breadth])
                    # print(f"  Adding {neighbor} with similarity {similarity}, Breadth: {new_breadth}")
    
    # Return the levels and the edge data
    return edge_data

def bfs_with_cumulative_similarity(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    queue = deque([(start_node, 0, 1.0)])  # Each element is a tuple: (node, breadth, cumulative_similarity)
    visited = set([start_node])            # Track visited nodes
    edge_data = []                         # To store [old_node, new_node, edge_similarity, cumulative_similarity, breadth]
    
    while queue:
        node, breadth, cumulative_similarity = queue.popleft()
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge
            edge_similarity = G[node][neighbor].get('sim', 0)
            
            # Calculate new cumulative similarity
            new_cumulative_similarity = cumulative_similarity * edge_similarity
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (new_cumulative_similarity >= similarity_threshold) or \
               (condition == '<=') and (new_cumulative_similarity <= similarity_threshold):
                
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_breadth = breadth + 1
                    queue.append((neighbor, new_breadth, new_cumulative_similarity))
                    edge_data.append([node, neighbor, new_cumulative_similarity, new_breadth])
    
    # Return the edge data
    return edge_data

In [7]:
i = 0
# Вершины, которые нужно исключить

# Создаем подграф, исключая определенные вершины

Make_a_class = pd.DataFrame(bfs_with_cumulative_similarity(G, i, similarity_threshold=0, condition='>='), columns=['id1', 'id2', 'sim', 'breadth'])
simularity = Make_a_class[['id2', 'sim', 'breadth']].rename({'id2': 'id'}, axis=1).set_index('id')

if train[train.id == i]['class'].values[0] == 'D':
    pred_classes.loc[simularity.index, 'D_sum'] += simularity.sim
    pred_classes.loc[simularity.index, 'D_count'] += 1 / simularity.breadth
elif train[train.id == i]['class'].values[0] == 'K':
    pred_classes.loc[simularity.index, 'K_sum'] += simularity.sim
    pred_classes.loc[simularity.index, 'K_count'] += 1 / simularity.breadth

print(i)
for i in range(1, 10000):

    Make_a_class = pd.DataFrame(bfs_with_cumulative_similarity(G, i, similarity_threshold=0, condition='>='), columns=['id1', 'id2', 'sim', 'breadth'])
    simularity = Make_a_class[['id2', 'sim', 'breadth']].rename({'id2': 'id'}, axis=1).set_index('id')

    if train[train.id == i]['class'].values[0] == 'D':
        pred_classes.loc[simularity.index, 'D_sum'] += simularity.sim 
        pred_classes.loc[simularity.index, 'D_count'] += 1 / simularity.breadth # (сделано) Потенциально для расчёта среднего лучше ещё делить на ширину типо, элемент не сильно далёкий на похожесть влияет сильнее чем далёкий.
        pred_classes.loc[simularity.index, 'D_squarred_difference'] += (simularity.sim - first_data.loc[simularity.index, 'K_mean'])**2
        pred_classes.loc[simularity.index, 'D_max'] = np.maximum(simularity.sim, pred_classes.loc[simularity.index, 'D_max'])
        pred_classes.loc[simularity.index, 'D_min'] = np.minimum(simularity.sim, pred_classes.loc[simularity.index, 'D_min'])
    elif train[train.id == i]['class'].values[0] == 'K':
        pred_classes.loc[simularity.index, 'K_sum'] += simularity.sim
        pred_classes.loc[simularity.index, 'K_count'] += 1 / simularity.breadth
        pred_classes.loc[simularity.index, 'K_squarred_difference'] += (simularity.sim - first_data.loc[simularity.index, 'K_mean'])**2
        pred_classes.loc[simularity.index, 'K_max'] = np.maximum(simularity.sim, pred_classes.loc[simularity.index, 'K_max'])
        pred_classes.loc[simularity.index, 'K_min'] = np.minimum(simularity.sim, pred_classes.loc[simularity.index, 'K_min'])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
pred_classes.to_csv('advanced_data_for_ml.csv')

In [3]:
pred_classes = pd.read_csv('first_data_for_ml.csv', index_col=0)
pred_classes

class       D_sum      D_count  D_mean       K_sum      K_count  K_mean
id                                                                            
0        NaN  777.614282  2975.333333     0.0  189.940639  1218.666667     0.0
1        NaN  379.111613  2749.833333     0.0  320.586678  1293.166667     0.0
2        NaN  464.316126  2942.166667     0.0  156.871831  1235.500000     0.0
3        NaN  294.214732  2791.833333     0.0  211.299259  1292.666667     0.0
4        NaN  820.776927  3009.000000     0.0  196.406639  1249.500000     0.0
...      ...         ...          ...     ...         ...          ...     ...
14995    NaN  892.810953  3019.000000     0.0  194.093758  1227.333333     0.0
14996    NaN  307.139350  2754.333333     0.0  273.660614  1323.500000     0.0
14997    NaN  367.227889  2820.833333     0.0  219.240044  1303.333333     0.0
14998    NaN  471.464040  2966.333333     0.0  215.806324  1298.000000     0.0
14999    NaN  460.232824  2657.666667     0.0  460.621661  1303.666667     0.0

[15000 rows x 7 columns]

In [4]:
pred_classes['D_mean'] = pred_classes.apply(lambda x: x.D_sum / x.D_count, axis=1)
pred_classes['K_mean'] = pred_classes.apply(lambda x: x.K_sum / x.K_count, axis=1)

In [5]:
pred_classes.max()

class              NaN
D_sum      1146.621238
D_count    3129.666667
D_mean        0.376765
K_sum       506.586030
K_count    1353.333333
K_mean        0.387644
dtype: float64

In [6]:
train = train.set_index('id')
train

class
id        
0        D
1        K
2        D
3        K
4        D
...    ...
9995     D
9996     K
9997     K
9998     D
9999     K

[10000 rows x 1 columns]

In [7]:
pred_classes.loc[train.index, 'class'] = train['class']
pred_classes

/tmp/ipykernel_46818/155788887.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pred_classes.loc[train.index, 'class'] = train['class']


class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                        
0         D  777.614282  2975.333333  0.261354  189.940639  1218.666667   
1         K  379.111613  2749.833333  0.137867  320.586678  1293.166667   
2         D  464.316126  2942.166667  0.157814  156.871831  1235.500000   
3         K  294.214732  2791.833333  0.105384  211.299259  1292.666667   
4         D  820.776927  3009.000000  0.272774  196.406639  1249.500000   
...     ...         ...          ...       ...         ...          ...   
14995   NaN  892.810953  3019.000000  0.295731  194.093758  1227.333333   
14996   NaN  307.139350  2754.333333  0.111511  273.660614  1323.500000   
14997   NaN  367.227889  2820.833333  0.130184  219.240044  1303.333333   
14998   NaN  471.464040  2966.333333  0.158938  215.806324  1298.000000   
14999   NaN  460.232824  2657.666667  0.173172  460.621661  1303.666667   

         K_mean  
id               
0      0.155859  
1      0.247908  
2      0.126970  
3      0.163460  
4      0.157188  
...         ...  
14995  0.158143  
14996  0.206770  
14997  0.168215  
14998  0.166261  
14999  0.353328  

[15000 rows x 7 columns]

# XGBoost!!!!!!


In [8]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import optuna
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

In [9]:
X, y = pred_classes.loc[0:9999].drop(columns=['class']), pred_classes.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = pred_classes.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

/tmp/ipykernel_46818/3718468339.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = pred_classes.loc[0:9999].drop(columns=['class']), pred_classes.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


In [10]:
X_train

D_sum      D_count    D_mean       K_sum      K_count    K_mean
id                                                                        
9254  550.314602  2966.666667  0.185499  156.915193  1250.833333  0.125449
1561  779.633056  3006.833333  0.259287  179.276239  1228.500000  0.145931
1670  416.741001  2812.666667  0.148166  240.413291  1273.166667  0.188831
6087  375.194550  2667.166667  0.140672  360.604932  1296.500000  0.278137
6669  370.999827  2717.500000  0.136522  367.311842  1309.500000  0.280498
...          ...          ...       ...         ...          ...       ...
5734  598.581313  2981.666667  0.200754  162.709051  1239.666667  0.131252
5191  356.226666  2708.500000  0.131522  336.309897  1300.000000  0.258700
5390  399.345593  2864.000000  0.139436  257.158756  1332.000000  0.193062
860   344.139729  2743.000000  0.125461  306.930235  1300.333333  0.236040
7270  339.821822  2820.333333  0.120490  210.030346  1289.500000  0.162877

[8000 rows x 6 columns]

In [11]:
X_val

D_sum      D_count    D_mean       K_sum      K_count    K_mean
id                                                                        
6252  306.561361  2901.833333  0.105644  124.872461  1254.166667  0.099566
4684  397.512132  2977.833333  0.133490  126.655937  1254.500000  0.100961
1731  406.343544  2914.666667  0.139413  188.356032  1288.166667  0.146220
4742  462.819300  2880.000000  0.160701  248.489143  1306.833333  0.190146
4521  422.512600  2968.500000  0.142332  113.768423  1214.500000  0.093675
...          ...          ...       ...         ...          ...       ...
6412  653.645498  2851.000000  0.229269  184.030467  1197.166667  0.153722
8285  593.017399  2975.166667  0.199322  148.278853  1222.666667  0.121275
7853  642.949395  2963.500000  0.216956  145.138081  1218.166667  0.119145
1095  728.996931  3075.500000  0.237034  188.516451  1268.833333  0.148575
6929  345.446411  2646.833333  0.130513  311.766382  1251.166667  0.249181

[2000 rows x 6 columns]

In [12]:
y_train

id
9254    1
1561    0
1670    0
6087    1
6669    1
       ..
5734    0
5191    1
5390    1
860     1
7270    0
Name: class, Length: 8000, dtype: int64

In [69]:
y_val

id
6252    0
4684    0
1731    0
4742    1
4521    1
       ..
6412    0
8285    0
7853    0
1095    0
6929    1
Name: class, Length: 2000, dtype: int64

In [15]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 6, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61247	validation_1-logloss:0.62786
[1]	validation_0-logloss:0.60986	validation_1-logloss:0.62526
[2]	validation_0-logloss:0.60731	validation_1-logloss:0.62278
[3]	validation_0-logloss:0.60481	validation_1-logloss:0.62035
[4]	validation_0-logloss:0.60237	validation_1-logloss:0.61798
[5]	validation_0-logloss:0.59998	validation_1-logloss:0.61564
[6]	validation_0-logloss:0.59764	validation_1-logloss:0.61334
[7]	validation_0-logloss:0.59536	validation_1-logloss:0.61116
[8]	validation_0-logloss:0.59312	validation_1-logloss:0.60900
[9]	validation_0-logloss:0.59092	validation_1-logloss:0.60688
[10]	validation_0-logloss:0.58876	validation_1-logloss:0.60485
[11]	validation_0-logloss:0.58665	validation_1-logloss:0.60282
[12]	validation_0-logloss:0.58458	validation_1-logloss:0.60088
[13]	validation_0-logloss:0.58253	validation_1-logloss:0.59902
[14]	validation_0-logloss:0.58054	validation_1-logloss:0.59716
[15]	validation_0-logloss:0.57860	validation_1-logloss:0.59532
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [70]:
f1_score(y_val, clf.predict(X_val))

np.float64(0.59375)

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, clf.predict(X_val))

0.779

D_sum      D_count    D_mean       K_sum      K_count    K_mean
id                                                                         
10000  801.396722  2991.333333  0.267906  180.319387  1215.500000  0.148350
10001  368.357546  2665.333333  0.138203  361.401813  1293.666667  0.279362
10002  667.923458  3008.500000  0.222012  162.752247  1246.166667  0.130602
10003  596.057236  2979.666667  0.200042  169.995673  1253.166667  0.135653
10004  760.345866  2982.666667  0.254922  180.251510  1234.000000  0.146071
...           ...          ...       ...         ...          ...       ...
14995  892.810953  3019.000000  0.295731  194.093758  1227.333333  0.158143
14996  307.139350  2754.333333  0.111511  273.660614  1323.500000  0.206770
14997  367.227889  2820.833333  0.130184  219.240044  1303.333333  0.168215
14998  471.464040  2966.333333  0.158938  215.806324  1298.000000  0.166261
14999  460.232824  2657.666667  0.173172  460.621661  1303.666667  0.353328

[5000 rows x 6 columns]

In [24]:
X_test.loc[:, 'class'] = clf.predict(X_test.drop(columns=['class']))
X_test

class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                        
10000     0  801.396722  2991.333333  0.267906  180.319387  1215.500000   
10001     1  368.357546  2665.333333  0.138203  361.401813  1293.666667   
10002     0  667.923458  3008.500000  0.222012  162.752247  1246.166667   
10003     0  596.057236  2979.666667  0.200042  169.995673  1253.166667   
10004     0  760.345866  2982.666667  0.254922  180.251510  1234.000000   
...     ...         ...          ...       ...         ...          ...   
14995     0  892.810953  3019.000000  0.295731  194.093758  1227.333333   
14996     1  307.139350  2754.333333  0.111511  273.660614  1323.500000   
14997     0  367.227889  2820.833333  0.130184  219.240044  1303.333333   
14998     0  471.464040  2966.333333  0.158938  215.806324  1298.000000   
14999     1  460.232824  2657.666667  0.173172  460.621661  1303.666667   

         K_mean  
id               
10000  0.148350  
10001  0.279362  
10002  0.130602  
10003  0.135653  
10004  0.146071  
...         ...  
14995  0.158143  
14996  0.206770  
14997  0.168215  
14998  0.166261  
14999  0.353328  

[5000 rows x 7 columns]

In [29]:
predict = X_test.replace({0: 'D', 1: 'K'}).reset_index()[['id', 'class']]

In [34]:
predict.to_csv('ml_predict1.csv', index=False)

In [101]:
predict['class'].value_counts()

class
D    4050
K     950
Name: count, dtype: int64

## Predict_proba


In [102]:
predo = pd.read_csv('ml_predict1.csv')
predo.loc[:, 'class'] = clf.predict_proba(X_test.drop(columns=['class'])).T[1]

In [103]:
predo['class'].apply(lambda x: x > 0.42).value_counts()

class
False    3879
True     1121
Name: count, dtype: int64

In [104]:
predo['class'] = predo['class'].apply(lambda x: x > 0.42)

In [105]:
predo['class'].value_counts()

class
False    3879
True     1121
Name: count, dtype: int64

In [106]:
predo.replace({False: "D", True: "K"}).to_csv("Ml_predict2.csv", index=False)

In [94]:
predo

id  class
0     10000  False
1     10001   True
2     10002  False
3     10003  False
4     10004  False
...     ...    ...
4995  14995  False
4996  14996   True
4997  14997  False
4998  14998  False
4999  14999   True

[5000 rows x 2 columns]

# Old Post-Processing


In [117]:
pred_classes.loc[:, 'class'] = 'D'
pred_classes.loc[pred_classes[(pred_classes.K_mean > pred_classes.D_mean) & (pred_classes.K_mean > 0.06)].index, 'class'] = 'K'
# pred_classes.loc[pred_classes[pred_classes.K_mean <= pred_classes.D_mean].index, 'class'] = 'D'

In [94]:
pred_classes.loc[pred_classes[pred_classes.K_mean > 0.063].index, 'class'] = 'K'
pred_classes.loc[pred_classes[pred_classes.K_mean <= 0.063].index, 'class'] = 'D'

In [124]:
pred_classes['class'].value_counts()

class
D    3846
K    1154
Name: count, dtype: int64

In [119]:
pred_classes

class       D_sum  D_count    D_mean       K_sum  K_count    K_mean
id                                                                       
10000     D  599.547474   6912.0  0.086740  137.069201   3088.0  0.044388
10001     K  284.393159   6912.0  0.041145  252.153682   3088.0  0.081656
10002     D  550.443102   6912.0  0.079636  131.340401   3088.0  0.042533
10003     D  479.328720   6912.0  0.069347  134.952938   3088.0  0.043702
10004     D  593.770419   6912.0  0.085904  144.352726   3088.0  0.046746
...     ...         ...      ...       ...         ...      ...       ...
14995     D  708.458124   6912.0  0.102497  159.778180   3088.0  0.051742
14996     D  173.067118   6912.0  0.025039  165.892871   3088.0  0.053722
14997     D  279.725701   6912.0  0.040470  161.493077   3088.0  0.052297
14998     D  338.116477   6912.0  0.048917  169.085405   3088.0  0.054756
14999     K  386.541760   6912.0  0.055923  381.013445   3088.0  0.123385

[5000 rows x 7 columns]

In [19]:
pred_classes[pred_classes['class'].isna()]

class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                         
0        NaN  777.614282  2975.333333  0.261354  189.940639  1218.666667   
1        NaN  379.111613  2749.833333  0.137867  320.586678  1293.166667   
2        NaN  464.316126  2942.166667  0.157814  156.871831  1235.500000   
3        NaN  294.214732  2791.833333  0.105384  211.299259  1292.666667   
4        NaN  820.776927  3009.000000  0.272774  196.406639  1249.500000   
...      ...         ...          ...       ...         ...          ...   
14995    NaN  892.810953  3019.000000  0.295731  194.093758  1227.333333   
14996    NaN  307.139350  2754.333333  0.111511  273.660614  1323.500000   
14997    NaN  367.227889  2820.833333  0.130184  219.240044  1303.333333   
14998    NaN  471.464040  2966.333333  0.158938  215.806324  1298.000000   
14999    NaN  460.232824  2657.666667  0.173172  460.621661  1303.666667   

         K_mean  
id               
0      0.155859  
1      0.247908  
2      0.126970  
3      0.163460  
4      0.157188  
...         ...  
14995  0.158143  
14996  0.206770  
14997  0.168215  
14998  0.166261  
14999  0.353328  

[15000 rows x 7 columns]

In [121]:
pred_classes[pred_classes['class'] == 0]

Empty DataFrame
Columns: [class, D_sum, D_count, D_mean, K_sum, K_count, K_mean]
Index: []

In [123]:
pred_classes.reset_index()[['id', 'class']].to_csv('bfs_third_version.csv', index=False)

In [12]:
pred_classes.loc[simularity.index].sort_index()

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10037    NaN  0.721931      1.0     0.0  3.601700      5.0     0.0
10067    NaN  1.404952      2.0     0.0  1.978038      3.0     0.0
10160    NaN  2.672333      4.0     0.0  1.475610      2.0     0.0
10198    NaN  3.382734      5.0     0.0  0.000000      0.0     0.0
10440    NaN  3.489978      5.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14655    NaN  3.739731      5.0     0.0  0.000000      0.0     0.0
14660    NaN  2.958026      4.0     0.0  0.608879      1.0     0.0
14736    NaN  3.641358      5.0     0.0  0.000000      0.0     0.0
14787    NaN  1.468872      2.0     0.0  0.716277      1.0     0.0
14801    NaN  4.909661      6.0     0.0  0.000000      0.0     0.0

[100 rows x 7 columns]

In [13]:
pred_classes[(pred_classes.D_count > 0) | (pred_classes.K_count > 0)]

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10000    NaN  2.395088      3.0     0.0  0.000000      0.0     0.0
10001    NaN  0.000000      0.0     0.0  1.625563      2.0     0.0
10002    NaN  3.488712      5.0     0.0  0.000000      0.0     0.0
10003    NaN  1.230105      2.0     0.0  0.000000      0.0     0.0
10004    NaN  4.015353      6.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14989    NaN  0.000000      0.0     0.0  2.077058      3.0     0.0
14990    NaN  1.986381      3.0     0.0  0.000000      0.0     0.0
14995    NaN  5.370993      8.0     0.0  0.000000      0.0     0.0
14998    NaN  1.234757      2.0     0.0  0.000000      0.0     0.0
14999    NaN  0.767606      1.0     0.0  3.901501      5.0     0.0

[3540 rows x 7 columns]

In [14]:
pred_classes

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10000    NaN  2.395088      3.0     0.0  0.000000      0.0     0.0
10001    NaN  0.000000      0.0     0.0  1.625563      2.0     0.0
10002    NaN  3.488712      5.0     0.0  0.000000      0.0     0.0
10003    NaN  1.230105      2.0     0.0  0.000000      0.0     0.0
10004    NaN  4.015353      6.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14995    NaN  5.370993      8.0     0.0  0.000000      0.0     0.0
14996    NaN  0.000000      0.0     0.0  0.000000      0.0     0.0
14997    NaN  0.000000      0.0     0.0  0.000000      0.0     0.0
14998    NaN  1.234757      2.0     0.0  0.000000      0.0     0.0
14999    NaN  0.767606      1.0     0.0  3.901501      5.0     0.0

[5000 rows x 7 columns]

In [16]:
pred_classes['class'].isna().sum()

np.int64(5000)

In [39]:
pred_classes[pred_classes.index.isin(simularity.index)]

class     D_sum  D_count  D_mean  K_sum  K_count  K_mean
id                                                             
10000    NaN  0.854674      0.0     0.0    0.0      0.0     0.0
10002    NaN  0.725023      0.0     0.0    0.0      0.0     0.0
10004    NaN  0.601816      0.0     0.0    0.0      0.0     0.0
10022    NaN  0.832458      0.0     0.0    0.0      0.0     0.0
10025    NaN  0.628172      0.0     0.0    0.0      0.0     0.0
...      ...       ...      ...     ...    ...      ...     ...
14912    NaN  0.713893      0.0     0.0    0.0      0.0     0.0
14939    NaN  0.878399      0.0     0.0    0.0      0.0     0.0
14947    NaN  0.802698      0.0     0.0    0.0      0.0     0.0
14950    NaN  0.611281      0.0     0.0    0.0      0.0     0.0
14965    NaN  0.669374      0.0     0.0    0.0      0.0     0.0

[438 rows x 7 columns]

In [71]:
pred_classes['class'].value_counts()

class
K    2611
D    2115
Name: count, dtype: int64

In [84]:
pred_classes[pred_classes['class'].isna()]

id class
34    10034   NaN
44    10044   NaN
80    10080   NaN
86    10086   NaN
125   10125   NaN
...     ...   ...
4972  14972   NaN
4974  14974   NaN
4978  14978   NaN
4991  14991   NaN
4994  14994   NaN

[274 rows x 2 columns]

In [73]:
pred_classes.isna()['class'].sum()

np.int64(274)

In [ ]:
pred_classes.to_csv('bfs_second_version.csv', index=False)

In [34]:
na = pred_classes[pred_classes.isna()['class']].id.values
na

array([10003, 10005, 10009, ..., 14994, 14996, 14997])